In [ ]:
# 1. Import libs

from fastai.tabular import *
from fastai.metrics import *

In [ ]:
# 2. procs \ cat_names \ cont_names \ dep_var 

# transformations
procs = [FillMissing, Categorify, Normalize]

# words
cat_names = []

# numbers
cont_names = ['SepalLengthCm','SepalWidthCm','PetalLengthCm', 'PetalWidthCm']

# target
dep_var = 'Species'

In [ ]:
# 3. data

data = (TabularList.from_df(df, procs=procs, cont_names=cont_names)
        .split_subsets(train_size=0.6,valid_size=0.4,seed=42)
        .label_from_df(cols=dep_var)
        .databunch())

In [ ]:
# 4. Learn

learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [ ]:
# 5. first fits

learn.fit_one_cycle(3)

In [ ]:
# 6. lr_find \ plot

learn.lr_find();
learn.recorder.plot()

In [ ]:
# 7. Save model before tuning

learn.save('stage-1')

In [ ]:
# 8. unfreeze / fit

learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3))

# -

In [ ]:
# lr find def

learn.unfreeze()
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
lr = learn.recorder.min_grad_lr
lr

In [ ]:
# lr find auto

def find_appropriate_lr(model:Learner, lr_diff:int = 15, loss_threshold:float = .05, adjust_value:float = 1, plot:bool = False) -> float:
    #Run the Learning Rate Finder
    model.lr_find()
    
    #Get loss values and their corresponding gradients, and get lr values
    losses = np.array(model.recorder.losses)
    assert(lr_diff < len(losses))
    loss_grad = np.gradient(losses)
    lrs = model.recorder.lrs
    
    #Search for index in gradients where loss is lowest before the loss spike
    #Initialize right and left idx using the lr_diff as a spacing unit
    #Set the local min lr as -1 to signify if threshold is too low
    r_idx = -1
    l_idx = r_idx - lr_diff
    while (l_idx >= -len(losses)) and (abs(loss_grad[r_idx] - loss_grad[l_idx]) > loss_threshold):
        local_min_lr = lrs[l_idx]
        r_idx -= 1
        l_idx -= 1

    lr_to_use = local_min_lr * adjust_value
    
    if plot:
        # plots the gradients of the losses in respect to the learning rate change
        plt.plot(loss_grad)
        plt.plot(len(losses)+l_idx, loss_grad[l_idx],markersize=10,marker='o',color='red')
        plt.ylabel("Loss")
        plt.xlabel("Index of LRs")
        plt.show()

        plt.plot(np.log10(lrs), losses)
        plt.ylabel("Loss")
        plt.xlabel("Log 10 Transform of Learning Rate")
        loss_coord = np.interp(np.log10(lr_to_use), np.log10(lrs), losses)
        plt.plot(np.log10(lr_to_use), loss_coord, markersize=10,marker='o',color='red')
        plt.show()
        
    return lr_to_use
find_appropriate_lr(model=learn)

In [ ]:
# lr find india

def find_lr(losses, lrs):
    import matplotlib.pyplot as plt

    losses_skipped = 5
    trailing_losses_skipped = 5
    losses = losses[losses_skipped:-trailing_losses_skipped]
    lrs = lrs[losses_skipped:-trailing_losses_skipped]

    n = len(losses)

    max_start = 0
    max_end = 0
    
    # finding the longest valley.
    lds = [1] * n

    for i in range(1, n):
        for j in range(0, i):
            if losses[i] < losses[j] and lds[i] < lds[j] + 1:
                lds[i] = lds[j] + 1
            if lds[max_end] < lds[i]:
                max_end = i
                max_start = max_end - lds[max_end]

    sections = (max_end - max_start) / 3
    final_index = max_start + int(sections) + int(sections/2) # pick something midway, or 2/3rd of the way to be more aggressive

    fig, ax = plt.subplots(1, 1)
    ax.plot(
        lrs,
        losses
    )
    ax.set_ylabel("Loss")
    ax.set_xlabel("Learning Rate")
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%.0e'))
    ax.plot(
        lrs[final_index],
        losses[final_index],
        markersize=10,
        marker='o',
        color='red'
    )

    plt.show()
    
    return lrs[final_index]
find_lr()